In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


# Data exploration

## Data extraction

In [ ]:
df_train = pd.read_csv('/kaggle/input/30-days-of-ml/train.csv', index_col = 'id')
df_test = pd.read_csv('/kaggle/input/30-days-of-ml/test.csv', index_col = 'id')

## Basic information

In [66]:
print('df_train:')
print(str(df_train.shape) + '\n' * 2 + 'df_test:')
print(df_test.shape)

df_train:
(300000, 25)

df_test:
(200000, 24)


In [67]:
print('df_train:')
print(str(df_train.head()) + '\n' * 2 + 'df_test:')
print(df_test.head())

df_train:
   cat0 cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9  ...     cont5     cont6  \
id                                                    ...                       
1     B    B    B    C    B    B    A    E    C    N  ...  0.400361  0.160266   
2     B    B    A    A    B    D    A    F    A    O  ...  0.533087  0.558922   
3     A    A    A    C    B    D    A    D    A    F  ...  0.650609  0.375348   
4     B    B    A    C    B    D    A    E    C    K  ...  0.668980  0.239061   
6     A    A    A    C    B    D    A    E    A    N  ...  0.686964  0.420667   

       cont7     cont8     cont9    cont10    cont11    cont12    cont13  \
id                                                                         
1   0.310921  0.389470  0.267559  0.237281  0.377873  0.322401  0.869850   
2   0.516294  0.594928  0.341439  0.906013  0.921701  0.261975  0.465083   
3   0.902567  0.555205  0.843531  0.748809  0.620126  0.541474  0.763846   
4   0.732948  0.679618  0.574844  0.346010

In [68]:
print('df_train:')
print(str(df_train.info()) + '\n' * 2 + 'df_test:')
print(df_test.info())

df_train:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000 entries, 1 to 499999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    300000 non-null  object 
 1   cat1    300000 non-null  object 
 2   cat2    300000 non-null  object 
 3   cat3    300000 non-null  object 
 4   cat4    300000 non-null  object 
 5   cat5    300000 non-null  object 
 6   cat6    300000 non-null  object 
 7   cat7    300000 non-null  object 
 8   cat8    300000 non-null  object 
 9   cat9    300000 non-null  object 
 10  cont0   300000 non-null  float64
 11  cont1   300000 non-null  float64
 12  cont2   300000 non-null  float64
 13  cont3   300000 non-null  float64
 14  cont4   300000 non-null  float64
 15  cont5   300000 non-null  float64
 16  cont6   300000 non-null  float64
 17  cont7   300000 non-null  float64
 18  cont8   300000 non-null  float64
 19  cont9   300000 non-null  float64
 20  cont10  300000 non-null  float64
 21  

In [69]:
print('df_train:')
print(str(df_train.describe()) + '\n' * 2 + 'df_test:')
print(df_test.describe())

df_train:
               cont0          cont1          cont2          cont3  \
count  300000.000000  300000.000000  300000.000000  300000.000000   
mean        0.527335       0.460926       0.490498       0.496689   
std         0.230599       0.214003       0.253346       0.219199   
min        -0.118039      -0.069309      -0.056104       0.130676   
25%         0.405965       0.310494       0.300604       0.329783   
50%         0.497053       0.427903       0.502462       0.465026   
75%         0.668060       0.615113       0.647512       0.664451   
max         1.058443       0.887253       1.034704       1.039560   

               cont4          cont5          cont6          cont7  \
count  300000.000000  300000.000000  300000.000000  300000.000000   
mean        0.491654       0.510526       0.467476       0.537119   
std         0.240074       0.228232       0.210331       0.218140   
min         0.255908       0.045915      -0.224689       0.203763   
25%         0.284188   

## Missing values information

In [70]:
df_train_missing_val_count = (df_train.isnull().sum())
df_test_missing_val_count = (df_test.isnull().sum())
print('df_train:')
print(str(df_train_missing_val_count) + '\n' * 2 + 'df_test:')
print(df_test_missing_val_count)

df_train:
cat0      0
cat1      0
cat2      0
cat3      0
cat4      0
cat5      0
cat6      0
cat7      0
cat8      0
cat9      0
cont0     0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
target    0
dtype: int64

df_test:
cat0      0
cat1      0
cat2      0
cat3      0
cat4      0
cat5      0
cat6      0
cat7      0
cat8      0
cat9      0
cont0     0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
dtype: int64


## Categorical variables information

In [126]:
object_cols = [col for col in df_train.columns if df_train[col].dtype == 'object']
for col in object_cols:
    print('df_train[' + col + ']:' + str(df_train[col].nunique()))
    print('df_test[' + col + ']:' + str(df_test[col].nunique()) + '\n')
good_label_cols = [col for col in object_cols if set(df_train[col]).issubset(set(df_test[col]))]
if object_cols == good_label_cols:
    print('No bad label columns.')
else:
    bad_label_cols = list(set(object_cols) - set(good_label_cols))
    print('Bad label column(s): ' + bad_label_cols)

df_train[cat0]:2
df_test[cat0]:2

df_train[cat1]:2
df_test[cat1]:2

df_train[cat2]:2
df_test[cat2]:2

df_train[cat3]:4
df_test[cat3]:4

df_train[cat4]:4
df_test[cat4]:4

df_train[cat5]:4
df_test[cat5]:4

df_train[cat6]:8
df_test[cat6]:8

df_train[cat7]:8
df_test[cat7]:8

df_train[cat8]:7
df_test[cat8]:7

df_train[cat9]:15
df_test[cat9]:15

No bad label columns.


# Data preparation

## Data split

In [114]:
y = df_train.target
X = df_train.drop(['target'], axis = 1)
print('X:')
print(str(X.shape) + '\n' * 2 + 'y:')
print(y.shape)

X:
(300000, 24)

y:
(300000,)


## Columns selecting

In [164]:
categorical_cols = [col for col in object_cols if (X[col].nunique() | X[col].nunique()) < 10]
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
cols_selected = categorical_cols + numerical_cols
X_train = X[cols_selected].copy()
X_test = df_test[cols_selected].copy()
print('columns selected: \n' + str(cols_selected) + '\n' * 2 + 'X_train:')
print(str(X_train.shape) + '\n' * 2 + 'X_test:')
print(X_test.shape)

columns selected: 
['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']

X_train:
(300000, 23)

X_test:
(200000, 23)


# Model building

## Parameters choosing

In [158]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

average_score = {}
for n_estimators in np.linspace(100, 500, 5, dtype = np.int):
    for learning_rate in [0.5, 0.1, 0.02, 0.004]:
        model = XGBRegressor(n_estimators = n_estimators, learning_rate = learning_rate, random_state = 0)
        categorical_transformer = Pipeline(steps = [
            ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
        ])
        preprocessor = ColumnTransformer(
            transformers = [
                ('cat', categorical_transformer, categorical_cols)
            ]
        )
        pipeline = Pipeline(steps = [
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        scores = -1 * cross_val_score(pipeline, X_train, y,
                                     cv = 5,
                                     scoring = 'neg_mean_squared_error')
        print(str(n_estimators) + ' and ' + str(learning_rate) + ': ' + str(scores.mean()))
        average_score[str(n_estimators) + '_' + str(learning_rate)] = scores.mean()        
best_parameters = min(average_score, key = average_score.get)
print(best_parameters)

100 and 0.5: 0.5580463784010146
100 and 0.1: 0.5551138373685054
100 and 0.02: 1.6090431774879363
100 and 0.004: 27.445148226259857
200 and 0.5: 0.559828494740378
200 and 0.1: 0.5557105971267697
200 and 0.02: 0.5731433025630472
200 and 0.004: 12.618226137132831
300 and 0.5: 0.5608682827570167
300 and 0.1: 0.55624935952842
300 and 0.02: 0.5550540142648097
300 and 0.004: 5.966608471563342
400 and 0.5: 0.5616117471644893
400 and 0.1: 0.5567544233445154
400 and 0.02: 0.5549001751988212
400 and 0.004: 2.982582336091599
500 and 0.5: 0.5621550419774909
500 and 0.1: 0.5571558416766169
500 and 0.02: 0.5550480116919937
500 and 0.004: 1.6439009428456324
400_0.02


## Data training

In [175]:
model_chosen = XGBRegressor(n_estimators = 400, learning_rate = 0.02, random_state = 0)
pipeline_chosen = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('model', model_chosen)
])
pipeline_chosen.fit(X_train, y)

## Data predicting

In [177]:
y_pred = pipeline_chosen.predict(X_test)
print(y_pred)

[8.139854 8.248583 8.291934 ... 8.213175 8.237408 8.239917]


# Data submission

In [181]:
output = pd.DataFrame({
    'id': X_test.index,
    'target': y_pred})
output.to_csv('my_submission.csv', index = False)